In [25]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("Auto_Reply_v3.db")

# Load Q&A pairs (assuming table name is 'All_Reviews')
query1 = 'SELECT "translation" AS Review, "Developer Reply Text" AS Reply FROM All_Reviews WHERE "Developer Reply Text" IS NOT NULL'
query2 = 'SELECT "translation" AS Review, "Reply" FROM "Auto Reply Hadith" WHERE "Reply" IS NOT NULL'
query3 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews WHERE "Reply" IS NOT NULL'
query4 = 'SELECT "translation" AS Review, "Reply" FROM Auto_Reply_Reviews_Al_Quran WHERE "Reply" IS NOT NULL'

df1 = pd.read_sql_query(query1, conn)
df2 = pd.read_sql_query(query2, conn)
df3 = pd.read_sql_query(query3, conn)
df4 = pd.read_sql_query(query4, conn)

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Close connection
conn.close()

# Check data
df.head()

# craete a csv file 
# df.to_csv("Auto_Reply.csv", index=False)

,Review,Reply
0,The recent edition does explain the source of ...,BarakAllahu feekum for your kind words and sup...
1,Alhamdulillah very good app but it would have ...,Assalamu 'alaykum dear user. We would love to ...
2,"Alhamdulillah, this is a amazing app.",BarakAllahu feekum for your kind words and sup...
3,Very good app. But the word-by-word translatio...,"Assalamu 'alaykum, exciting news! French Word-..."
4,The app is really great!,Assalamu 'alaykum dear user. We would love to ...


In [26]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [27]:
# Load sentence transformer model for embeddings
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Compute embeddings for all reviews
df["Embedding"] = df["Review"].apply(lambda x: model.encode(x, convert_to_numpy=True))

In [28]:
def retrieve_reply(query, df, model, top_k=1):
    """Retrieve the most relevant reply for a given review query."""
    query_embedding = model.encode(query, convert_to_numpy=True)
    
    # Compute similarity
    similarities = np.array([cosine_similarity([query_embedding], [emb])[0][0] for emb in df["Embedding"]])
    
    # Get the top-k most similar responses
    top_idx = similarities.argsort()[-top_k:][::-1]
    return df.iloc[top_idx]["Reply"].values[0]

In [29]:
# Example query
query_review = "This app is great, but I am facing some issues."
retrieved_reply = retrieve_reply(query_review, df, model)

retrieved_reply

'BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!'

In [30]:
# Example query
query_review = "This app is not good, I am facing some issues."
retrieved_reply = retrieve_reply(query_review, df, model)

retrieved_reply

'BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!'

In [31]:
# Example query
query_review = "Bad App"
retrieved_reply = retrieve_reply(query_review, df, model)

retrieved_reply

"Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran."